# 0. One-time data save to Gdrive

In [1]:
# Mount drive - needed for GBQ creds
from google.colab import drive
drive.mount('/content/drive')

import pickle
df = pickle.load(open('/content/drive/MyDrive/GitHub/TDmatch/data/imdb/imdb_reviews_1000film.df','rb'))
df.to_csv('/content/drive/MyDrive/GitHub/simplER/data/imdb_reviews.csv')

Mounted at /content/drive


# 1. NER w/ flair

In [ ]:
!pip install flair
import pandas as pd
from flair.nn import Classifier
from flair.data import Sentence
from flair.splitter import SegtokSentenceSplitter

In [23]:
# -------------
# Load model
# -------------

tagger = Classifier.load('ner-ontonotes-large') #ner-large

# example text with many sentences
text = ["dark  yes  complex  ambitious. christopher nolan and his co-writer jonathan nolan deserve a standing ovation. i don't usually go for loud movies filled with mindless gore and violence. the dark knight is certainly loud and violent but it's not mindless. it has depth and soul. even the joker  in an extraordinary creation by heath ledger  is deeply human. the natural petulance of christian bale makes his ego and alter ego the most fascinating and complex of all film superheroes. part of the genius of this movie is that batman himself  in screen time  is not really the lead. my attention was captivated by heath ledger and he determines and inspires the breathtaking atmosphere that envelopes gotham as well as us. the aplomb of christopher nolan as a director is mind blowing and his secret  i believe  is his obvious respect for his audience. what he's done is to elevate a popular genre into shakesperean proportions. bravo!"
,"'the godfather' is the pinnacle of flawless films! the first time i viewed 'the godfather' i was in my early teens and it was the most astounding film i had ever seen  and has since then stood as my all-time favourite film. it is due to this that i have been looking forward to writing a review of this unforgettable classic. so let's start from the beginning. the film opens to four words  'i believe in america'  it's crazy to think that this simple line has become a resonant quote solely due to the impact it made on the entrance to the film's ""threshold"". this is just one of the many renowned quotes that litter the film  and believe me  there are a lot. after the first take we are then absorbed into the life of vito corleone  brilliantly portrayed by the oscar- winning performance of marlon brando. vito is a feared man  he is a criminal  he is a mafioso  but above all he is a respected family man  his three sons are depicted by three excellent actors  james caan  john cazale and al pacino as well as his adopted child tom hagen  played by robert duvall. the film follows vito as he attempts to transfer his crime empire to his reluctant son  michael. with some of the most graphic and gruesome death scenes to have ever been seen in the 1970's film industry (including a certain horse's head)  'the godfather' epitomises how violence can be used effectively within a film. the corleone's are some of the greatest antiheroes to have ever been seen on screen  whilst they are villains  the audience will refuse to accept that fact  coppola does something simple and audacious  he takes the guilt out of organised crime. a film rife with beautiful cinematography  memorable musical scores and well-paced action and drama. overall  the godfather is one of hollywood's greatest critical and commercial successes that gets everything right; a gangster flick that is overflowing with life  rich with emotion and subtle acting  and further blessed with amazing direction from francis ford coppola. arguably the most unforgettable masterpiece to have ever been made."
]

In [33]:
# -----------------------------
# v1: split & predict together
# -----------------------------

ner_results = []

for i, ele in enumerate(text):

  # use splitter to split text into list of sentences
  splitter = SegtokSentenceSplitter()
  sentences = splitter.split(ele)

  # predict tags for sentences
  tagger.predict(sentences)

  for sentence in sentences:
    sentence_results = [{'id': i, 'text': x.data_point.text, 'entity_detected': x.value, 'score': x.score} for x in sentence.get_labels('ner')]

    if sentence_results:
      ner_results += sentence_results

In [35]:
ner_df = pd.DataFrame(ner_results) # keep all entities found just in case for doc retrieval -> do cleaning and majority voting w/ SQL

In [41]:
# -------------------------------
# alt: split & predict separate
# -------------------------------
split_list = []

for i, ele in enumerate(text):

  # use splitter to split text into list of sentences
  splitter = SegtokSentenceSplitter()
  sentences = splitter.split(ele)

  split_list.append({'id': i, 'input_text': ele, 'sentences': sentences})

In [43]:
split_df = pd.DataFrame(split_list)

In [48]:
ner_results_alt = []

for index, row in split_df.iterrows():

  sentences = row['sentences']
  
  # predict tags for sentences
  tagger.predict(sentences)

  for sentence in sentences:
    sentence_results = [{'id': row['id'], 'text': x.data_point.text, 'entity_detected': x.value, 'score': x.score} for x in sentence.get_labels('ner')]

    if sentence_results:
      ner_results_alt += sentence_results

In [50]:
pd.DataFrame(ner_results_alt)

,id,text,entity_detected,score
0,0,christopher nolan,PERSON,0.999998
1,0,jonathan nolan,PERSON,0.999998
2,0,the dark knight,WORK_OF_ART,0.999988
3,0,heath ledger,PERSON,0.999997
4,0,christian bale,PERSON,0.999998
5,0,batman,PERSON,0.999466
6,0,heath ledger,PERSON,0.999998
7,0,gotham,GPE,0.999980
8,0,christopher nolan,PERSON,0.999998
9,1,'the godfather,WORK_OF_ART,0.944807
